## Import libs and config data

In [ ]:
import praw
from datetime import datetime
import configparser
import time
from tqdm import tqdm
import pandas as pd

config = configparser.ConfigParser()
config.read("config.ini")

reddit = praw.Reddit(
    client_id=config.get("reddit", "client_id"),
    client_secret=config.get("reddit", "client_secret"),
    user_agent=config.get("reddit", "user_agent"),
    check_for_async=False,
)

## Load data from Reddit

### Top posts

In [2]:
subreddit = reddit.subreddit("politics")

comms_top = []

# Получаем список самых обсуждаемых тем за день
top_submissions = subreddit.top(time_filter="day", limit=75)

# Проходим по всем обсуждаемым темам
for submission in tqdm(top_submissions):
    # Проверяем, что тема была создана за последние 24 часа
    if submission.created_utc > time.time() - 24 * 60 * 60:
        submission.comments.replace_more(
            limit=0
        )  # Загружаем только те комментарии, которые алгоритмы Reddit считают самыми важными
        for comment in submission.comments.list():
            comms_top.append(comment.body)

len(comms_top)

75it [02:02,  1.64s/it]


16992

### Hot posts

In [3]:
comms_hot = []

# Получаем список самых обсуждаемых тем за день
hot_submissions = subreddit.hot(limit=75)

# Проходим по всем обсуждаемым темам
for submission in tqdm(hot_submissions):
    # Проверяем, что тема была создана за последние 24 часа
    if submission.created_utc > time.time() - 24 * 60 * 60:
        submission.comments.replace_more(
            limit=0
        )  # Загружаем только те комментарии, которые алгоритмы Reddit считают самыми важными
        for comment in submission.comments.list():
            comms_hot.append(comment.body)
len(comms_hot)

75it [01:54,  1.53s/it]


16046

### Save data to csv

In [4]:
def save_data(comments, top_or_hot):
    # Сохраним данные в csv
    current_date = datetime.now().strftime("%Y_%m_%d")
    current_time = datetime.now().strftime("%H_%M_%S")
    df = pd.DataFrame(data=comments, columns=["comments"])
    df["date"] = current_date
    df["time"] = current_time
    df.to_csv(
        f"original_data/{top_or_hot}/{subreddit.display_name}_{current_date}.csv",
        index=False,
    )

In [5]:
save_data(comms_top, "top")
save_data(comms_hot, "hot")